In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, accuracy_score
import joblib
import json
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier


try:
    df = pd.read_csv(r"C:\Users\johan\OneDrive - Universidad del Norte\Escritorio\MachineLearning\heart-disease-mlops\heart.csv")
except FileNotFoundError:
    print("Error: No se encontró 'heart.csv'.")


df_encoded = pd.get_dummies(df, drop_first=True)

X = df_encoded.drop("HeartDisease", axis=1)
y = df_encoded["HeartDisease"]

cols = X.columns.tolist()
with open("training_columns.json", "w") as f:
    json.dump(cols, f)

print(f"✅ Guardadas {len(cols)} columnas en training_columns.json")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Dimensiones de entrenamiento:", X_train.shape)
print("Dimensiones de prueba:", X_test.shape)

✅ Guardadas 15 columnas en training_columns.json
Dimensiones de entrenamiento: (734, 15)
Dimensiones de prueba: (184, 15)


In [21]:
def train_pipeline(model, param_grid, X_train, y_train, X_test, y_test):
    pipe = Pipeline([
        ("scaler", MinMaxScaler()),
        ("clf", model)
    ])

    grid = GridSearchCV(
        pipe,
        param_grid=param_grid,
        cv=5,
        scoring="roc_auc",
        n_jobs=-1
    )
    grid.fit(X_train, y_train)

    y_pred = grid.predict(X_test)
    
    if hasattr(grid.best_estimator_, "predict_proba"):
        y_prob = grid.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(y_test, y_prob)
    else:
        try:
            y_prob_decision = grid.decision_function(X_test)
            auc = roc_auc_score(y_test, y_prob_decision)
        except AttributeError:
            auc = np.nan

    acc = accuracy_score(y_test, y_pred)

    print(f"--- Modelo: {model.__class__.__name__} ---")
    print(f"Mejor AUC: {auc:.4f}")
    print(f"Mejor Acc: {acc:.4f}")
    print(f"Mejores Parámetros: {grid.best_params_}\n")

    return {
        "modelo": model.__class__.__name__,
        "mejor_auc": auc,
        "mejor_acc": acc,
        "mejores_param": grid.best_params_,
        "mejor_modelo": grid.best_estimator_
    }



In [22]:
modelos = [
    {
        "modelo": LogisticRegression(max_iter=1000, solver='liblinear'),
        "param_grid": {
            "clf__penalty": ['l1', 'l2'],
            "clf__C": [0.01, 0.1, 1, 10]
        }
    },
    {
        "modelo": RandomForestClassifier(random_state=42),
        "param_grid": {
            "clf__n_estimators": [100, 200],
            "clf__max_depth": [5, 10, None]
        }
    },
    {
        "modelo": KNeighborsClassifier(),
        "param_grid": {
            "clf__n_neighbors": [3, 5, 7, 9]
        }
    },
    {
        "modelo": XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'),
        "param_grid": {
            "clf__n_estimators": [100, 200],
            "clf__learning_rate": [0.01, 0.1],
            "clf__max_depth": [3, 5]
        }
    },
    {
        "modelo": GaussianNB(),
        "param_grid": {
            "clf__var_smoothing": [1e-9, 1e-8, 1e-7] 
        }
    },
    {
        "modelo": RidgeClassifier(random_state=42),
        "param_grid": {
            "clf__alpha": [0.1, 1.0, 10.0]
        }
    }
]



In [23]:
resultados = []

for m in modelos:
    r = train_pipeline(m["modelo"], m["param_grid"], X_train, y_train, X_test, y_test)
    resultados.append(r)

df_resultados = pd.DataFrame(resultados).sort_values(by="mejor_auc", ascending=False, na_position='last')

print("=== Resultados Finales de la Competición ===")
print(df_resultados[['modelo', 'mejor_auc', 'mejor_acc', 'mejores_param']])

--- Modelo: LogisticRegression ---
Mejor AUC: 0.9257
Mejor Acc: 0.8533
Mejores Parámetros: {'clf__C': 1, 'clf__penalty': 'l2'}

--- Modelo: RandomForestClassifier ---
Mejor AUC: 0.9300
Mejor Acc: 0.8587
Mejores Parámetros: {'clf__max_depth': 5, 'clf__n_estimators': 100}

--- Modelo: KNeighborsClassifier ---
Mejor AUC: 0.9164
Mejor Acc: 0.8370
Mejores Parámetros: {'clf__n_neighbors': 9}

--- Modelo: XGBClassifier ---
Mejor AUC: 0.9334
Mejor Acc: 0.8804
Mejores Parámetros: {'clf__learning_rate': 0.1, 'clf__max_depth': 3, 'clf__n_estimators': 100}

--- Modelo: GaussianNB ---
Mejor AUC: 0.9302
Mejor Acc: 0.8587
Mejores Parámetros: {'clf__var_smoothing': 1e-09}

--- Modelo: RidgeClassifier ---
Mejor AUC: 0.9239
Mejor Acc: 0.8370
Mejores Parámetros: {'clf__alpha': 0.1}

=== Resultados Finales de la Competición ===
                   modelo  mejor_auc  mejor_acc  \
3           XGBClassifier   0.933366   0.880435   
4              GaussianNB   0.930210   0.858696   
1  RandomForestClassifier  

c:\Users\johan\miniconda3\envs\ml_venv\lib\site-packages\xgboost\core.py:158: UserWarning: [11:13:29] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [24]:
import os
import json
import joblib 

best_model_pipeline = df_resultados.iloc[0]["mejor_modelo"]
best_model_name = df_resultados.iloc[0]["modelo"]

# --- Ambas rutas deben apuntar a la raíz (../) ---
output_path_model = "../model.joblib"
output_path_cols = "../training_columns.json"

# Guardar el modelo
joblib.dump(best_model_pipeline, output_path_model)

# Guardar las columnas
with open(output_path_cols, "w") as f:
    json.dump(cols, f)

print(f"\n✅ Modelo exportado en: {os.path.abspath(output_path_model)}")
print(f"✅ Columnas exportadas en: {os.path.abspath(output_path_cols)}")


✅ Modelo exportado en: c:\Users\johan\OneDrive - Universidad del Norte\Escritorio\MachineLearning\heart-disease-mlops\model.joblib
✅ Columnas exportadas en: c:\Users\johan\OneDrive - Universidad del Norte\Escritorio\MachineLearning\heart-disease-mlops\training_columns.json
